In [39]:
%load_ext autoreload
%autoreload 2
import os, pickle, csv, itertools, shutil, random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torchvision, torch
from scipy import stats

%matplotlib inline
%config InlineBackend.figure_format='retina'

import utils.plot_data as plot
import models.transforms as transforms
import algorithms.teaching_algs as algs
import evals.teaching_evals as evals

def get_ci(samples, confidence=0.95):  return 2 * stats.sem(samples) * stats.t.ppf((1 + confidence) / 2., len(samples)-1)
import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
train = torchvision.datasets.ImageFolder(f"data/datasets/bm/train", transform=transforms.shrink(63))
test = torchvision.datasets.ImageFolder(f"data/datasets/bm/test", transform=transforms.shrink(63))
x_train = np.array([x[0].numpy() for x in train])
x_test = np.array([x[0].numpy() for x in test])
y_train = np.array([x[1] for x in train])
y_test = np.array([x[1] for x in test])

In [9]:
lpips_path = "data/dist/lpips/bm"
# lpips_train = pickle.load(open(f"{lpips_path}/lpips.alex.train.pkl","rb"))
lpips_test = pickle.load(open(f"{lpips_path}/lpips.alex.test.pkl","rb"))

### random examples

In [44]:
n_trials = 1000

random_scores = np.array([evals.get_knn_score_lpips(lpips_test, algs.random(X, 10), y_train, y_test) for _ in range(n_trials)])
random_ci = get_ci(random_scores)
random_scores = random_scores.mean()
random_scores, random_ci

(0.6456000000000001, 0.013490983347214778)

### protodash

In [45]:
X = np.array([x.flatten() for x in x_train])
teaching_idx = algs.protodash(X, 10)
evals.get_knn_score_lpips(lpips_test, teaching_idx, y_train, y_test, k=1)

0.825

### ?